In [ ]:
from mlrepricer import match, setup, helper
import altair as alt
import numpy as np
import pandas as pd
import warnings
import datetime as dt
alt.data_transformers.enable('default', max_rows=1000000)

In [ ]:
inputdf = helper.load_dataframe('alldata')

In [ ]:
main = inputdf[inputdf.instock==1].reset_index()

In [ ]:
main['time_asin'] = main.time_changed.dt.strftime(date_format="%Y-%m-%d %H:%M:%S:%f") +'_'+ main.asin
main = main[['time_asin','price', 'isprime', 'isbuyboxwinner']]

In [ ]:
idx = main.reset_index().groupby(['time_asin'])['isbuyboxwinner'].idxmax()
# we need those index values twice
# m = lookup['index'].values
lookup = main.iloc[idx].set_index('time_asin')
assert (lookup['isbuyboxwinner']==1).all
lookup.drop('isbuyboxwinner', axis=1, inplace=True)

In [ ]:
removedmax = main.iloc[~idx].set_index('time_asin')

In [ ]:
result = pd.merge(removedmax, lookup, how='left', left_index=True, right_index=True, suffixes=('','_buybox'))

In [ ]:
# we can't and dont want to compare prime and prime. Same forn nonprime and nonprime.
filter1 = result[result.isprime!=result.isprime_buybox]

In [ ]:
conditions = [filter1.isprime==1, filter1.isprime_buybox==1]
choices = [filter1.price, filter1.price_buybox]

filter1['prime_price'] = np.select(conditions, choices, None)

conditions = [filter1.isprime==0, filter1.isprime_buybox==0]
choices = [filter1.price, filter1.price_buybox]

filter1['nonprime_price'] = np.select(conditions, choices, None)

In [ ]:
price = alt.Chart(filter1).mark_point(opacity=0.4).encode(
    y=alt.Y('nonprime_price'),
    x=alt.X('prime_price', scale=alt.Scale(zero=False)),
    color=('isprime_buybox:N')).interactive()
x = np.arange(25)
# here you should but in your function from the statsmodel
data = pd.DataFrame({'nonprime_price': x*0.859-0.5670,
                     'prime_price': x})

regr = alt.Chart(data).mark_line().encode(
    x='prime_price',
    y='nonprime_price'
)

In [ ]:
price + regr

In [ ]:
filter1[filter1.isbuyboxwinner==1]